# Jupyter notebook sample

## Parameters and dataset

In [ ]:
import os
import ipywidgets as widgets
from IPython.core.display import Markdown
from IPython.display import display

vehicle_capacity = 30

datasets = []

for root, dirs, files in os.walk("./Dataset"):
    for file in files:
        if file.lower().endswith('.txt'):
            datasets.append(os.path.join(root, file))

filename = os.environ['CURRENT_DATASET_FILEPATH'] if 'CURRENT_DATASET_FILEPATH' in os.environ else datasets[0]

# Create dropdown
dropdown = widgets.Dropdown(
    options=datasets,
    value=filename,
    description='Dataset:',
    disabled=False
)

# Dropdown event handler
def on_change(change):
    global filename
    filename = change['new']
    os.environ['CURRENT_DATASET_FILEPATH'] = change['new']
    
# Connect the function to the dropdown
dropdown.observe(on_change, "value")

# Display the dropdown
display(Markdown("#### Select Dataset:"))
display(dropdown)
display(Markdown("<br> Re execute notebook after selecting dataset"))


## Fetch and parse data

In [ ]:
print(filename)

with open(filename) as file:
    lines = file.readlines()

lines = lines[10:-1]

for i in range(len(lines)):
    lines[i] = lines[i].strip().split()
    lines[i] = list(map(int, lines[i]))

lines

## Create dataframe

In [ ]:
import pandas as pd

locations = pd.DataFrame(lines, columns=["id", "x", "y", "demand", "ready_time", "due_date", "service"])

locations

In [ ]:
import matplotlib.pyplot as plt

# Create the scatter plot
plt.figure(figsize=(10, 6))

# Draw warehouse
plt.scatter(locations['x'][0], locations['y'][0], c='red', alpha=0.5)

# Draw customers
plt.scatter(locations['x'][1:], locations['y'][1:], c='blue', alpha=0.5)

# Add labels for each point
for i, row in locations.iterrows():
    plt.annotate(row['id'],
                 (row['x'], row['y']),
                 xytext=(5, 5),  # 5 points offset
                 textcoords='offset points',
                 fontsize=10,
                 ha='left')  # horizontal alignment



# Customize the plot
plt.title('Delivery locations (warehouse is in red)')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


## Turn into location objects

In [ ]:
from src.Location import Location

locations = [Location(**record) for record in locations.to_dict('records')]

locations

## Compute VRP solution using nearest neighbor

In [ ]:
from src.Route import Route

route = Route(warehouse=locations[0], customers=[])

current = locations.pop(0)

while locations:
    current, locations = current.find_closest(locations)
    route.customers.append(current)

route.plot(figsize=(10, 10))